In [28]:
import requests
from bs4 import BeautifulSoup

url = "https://twitter.com/search?q=depressed&src=typed_query"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

print(soup.find_all('p')[0].get_text())

if soup.find_all('p')[0].get_text() == "We've detected that JavaScript is disabled in your browser. Would you like to proceed to legacy Twitter?" :
    print("gotcha")
else:
    print("fail")

We've detected that JavaScript is disabled in your browser. Would you like to proceed to legacy Twitter?
gotcha


In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd

In [2]:
class PostsFromTwitter:

    def __init__(self, query,fromDate,toDate):
        
        ''' URL examples
        https://mobile.twitter.com/search?q=depressed%2Canxiety&src=typed_query
        https://mobile.twitter.com/search?q=%23depressed%23anxiety&src=typed_query
        https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3A2020-09-19
        '''
        self.query = quote(query.strip())
        self.fromDate = fromDate
        self.toDate = toDate
        self.after_part = (
            f"&src=typed_query"
        )
        self.url = f"https://mobile.twitter.com/search?q={self.query}+since%3A{self.fromDate}+until%3A{self.toDate}"
        self.url += self.after_part
        self.tweets = []

    def scrape_tweets(self):
        
        reloadCount = 35
        
        while reloadCount > 0 :
            #print("URL to be scraped :: ", self.url)
            content = requests.get(self.url)
            soup = BeautifulSoup(content.text, "html.parser")
            tweet_divs = soup.select("#main_content")[0].select(".tweet")
#             print(tweet_divs)

#             firstId = int(tweet_divs[0].find("div", {"class": "tweet-text"})['data-id'])
#             lastId = int(tweet_divs[19].find("div", {"class": "tweet-text"})['data-id'])
#             print(firstId)
#             print(lastId)

            for tweet in tweet_divs:
                handle = tweet.find("div", {"class": "username"}).text.replace("\n", " ").strip()
                post = tweet.find("div", {"class": "tweet-text"}).text.replace("\n", " ").strip()
                tweetId = int(tweet.find("div", {"class": "tweet-text"})['data-id'])
                timestamp = tweet.find("td", {"class": "timestamp"}).find('a').text.replace("\n", " ").strip()
                self.tweets.append([tweetId,handle,post,timestamp])


            loadMoreButton = soup.select("#main_content")[0].select(".w-button-more")
            try:
                nextCursorUrl = "https://mobile.twitter.com"+loadMoreButton[0].find('a')['href']
            except IndexError:
                print("Oops!  list index out of range " , reloadCount)
                break
            self.url = nextCursorUrl
            
            reloadCount = reloadCount - 1
        
        print(self.url)
        return self.tweets


x = PostsFromTwitter("#depressed","2020-01-01","2020-09-19")
final_list = x.scrape_tweets()

Oops!  list index out of range  7
https://mobile.twitter.com/search?q=%23depressed%20since%3A2020-01-01%20until%3A2020-09-19&next_cursor=thGAVUV0VFVBaAwKH1t7rqmSQWjICwya3a4qMkEhjIBQQ9VDthRbvUAAAAAAAAFXwAAAAHAAAAVoxOEhbxlCGDx5RJhMIhGK4oUHqxiGXywtS9pyEQ6BzC-TKuAC-AgCpRLTjNpRoEIvwHJOFdgzJhAk7Rhb8SxHjxgyPHLysnZAIkKZAY5ZI-UKpXVkAhQkOqQgnTqoapDk2vAYQsF-bZQSWFMhbn7QUPUwBggdLAJpAN1sIHPAXTGNrgEpiLJso7WMPypLmT-EgKUzoARRE0UFymaAmTmBCbEVDUNCV8EQOQqCoaEpkozWYpxHDVFFGiCUjERMLQUYQAgmVYFC80t7CXHkCFMCSSISHMHUHECFxgpaJ5pQ5QYbE8ouVnRFKcWWVgiIpDsSiBAoAXMPHrFMQTEbKRmDCELUwR_SVUSo5SgxA0d4JYYAuMoOYKwCpMJl4xMriZLBjEijlGBrUO1OiOrRKiWgSXECiHG8frwIdojBcKFgIAWAUjEuBDDmGd7HAK5ARTdoIFVGCDQBKlMY0nuRpApGTwNRK44wgdAIAYCAbTrbioECyfCSBIyAYVsrBiEgL2aYgRpIAxfWRuCQEMIIARAyyAhKZ9Q2VpEYuHiwtJJi5UAGZFyozl2RWdGSeMpQ7XgoCGus7BoIkAOB8o4HNOcBEPHMQK6SiJOQFKQQgypZibWmKwMkUJAMF0IUKwERpzOA5hBktlKiXVrLgkH4oSoAH4ISgPTAFi4UB6gZCopEoQOTReYwsiQKaV4xDDG64cWIo8ALxASjWjIrfIWQIeRIrUyK55ilhS2ggmEDniFWdoB6ZbyzzJFgOaT00JUBhSJhgVAIGRhmlQxUR2qAkG6

In [3]:
print(final_list[len(final_list)-1])
len(final_list)

[1304307667502903296, '@lifeupeducation', 'For real   #help #love #depression #sad #life #sadness #follow #depressed #anxiety #mentalhealth #instagram #support #l #cry #insta #me #art #broken #hope #god #memes #crying #sadquotes #mood #rescue #alone #like #funny #tears #bhfyp pic.twitter.com/HfNLEWuy3y', 'Sep 10']


395

In [4]:
df = pd.DataFrame(final_list,columns=['tweet_id','@','tweet','date'])
df

,tweet_id,@,tweet,date
0,1306491288850706433,@Austin7397,It hurts. It hurts a lot. But I'll keep it to ...,Sep 17
1,1307095401883893764,@Zee_Kamboh_25,#Depressed...? Earphones..! Volume up..! And i...,20h
2,1306679415548641281,@I_Wicked1,Why won't you stop me..... #depressed,Sep 17
3,1306760408745811968,@PHMIWTKM,Some times you just gotta walk away #hurt #dep...,Sep 17
4,1306503153936859136,@DoctorsDanger1,Are you #depressed? Check your #gut_health. us...,Sep 17
...,...,...,...,...
390,1305475256828735489,@slowlydyingrns,Part 2️⃣ Ever wonder why it’s so hard to let g...,Sep 14
391,1305929713819570177,@thercftasued,Day 046/100 God is a mighty Healer #100daysw...,Sep 15
392,1306873092539445248,@themeeraki,Best Natural Supplement for Depression #depre...,Sep 18
393,1304292646668566528,@ConnectForeign,Being correct means zero. Being helpful means ...,Sep 10
